# Sortiert nach Session (Neues Dataframe)
Hier werden die Daten transformiert und in ein neues Dataframe umgewandelt. Dabei lässt sich eine Session in einer Row darstellen.

### Allgemeines Setup


In [2]:
import numpy as np
import matplotlib.pyplot as plt
import json
import pandas as pd
from pandas import read_csv
from sklearn.utils import shuffle
from datetime import datetime

/Users/ingaglotzbach/opt/anaconda3/lib/python3.8/site-packages/pandas/core/computation/expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


#### 1.0 Lade Daten

In [3]:
df_data = pd.read_pickle('../resources/CleanedDataWithIds.pkl') 

#### 1.1 Dateien werden nach session und date sortiert

In [4]:
df_replaced_together = df_data.sort_values(by=['session','date'], ascending=[False, True])

In [5]:
#get all new id numbers 
sessionId = df_replaced_together['session']
df_replaced_together['is_duplicated'] = df_replaced_together.duplicated('session')
#86 ids auf Seite Einfach
numbers1 = set(sessionId)
numbers = list(numbers1)

print(numbers)

[515, 519, 524, 528, 554, 569, 571, 573, 574, 579, 583, 594, 595, 605, 608, 102, 618, 110, 623, 112, 626, 116, 120, 633, 634, 127, 640, 136, 650, 138, 141, 144, 657, 660, 157, 158, 161, 680, 691, 695, 698, 701, 190, 703, 194, 196, 199, 201, 205, 211, 728, 226, 227, 743, 749, 239, 242, 243, 245, 758, 759, 253, 254, 260, 782, 783, 273, 279, 797, 798, 286, 288, 285, 802, 290, 297, 813, 823, 313, 314, 833, 324, 837, 845, 338, 851, 852, 345, 867, 869, 359, 360, 368, 371, 889, 891, 896, 384, 898, 388, 389, 905, 395, 398, 408, 921, 922, 413, 927, 929, 419, 934, 423, 940, 428, 949, 438, 439, 440, 960, 454, 457, 461, 976, 472, 992, 480, 994, 995, 483, 504, 510]


#### 1.2 Hier wird ein neues Dataframe kreiert

In [6]:
session = pd.DataFrame(numbers, columns=["session"])
arrMouseover = []
arrMouseout = []
arrClick = []
arrBlur = []
arrFocus = []
arrKeydown = []
arrDate = []
arrHeight = []
arrWidth = []
arrSiteName = []
arrClientX = []
arrClientY = []
arrCount = []

for i in range(len(numbers)):
    new = df_replaced_together.loc[df_replaced_together['session'] == numbers[i]]
    date = (new['date'].iloc[-1] - (new['date'].iloc[0])).total_seconds()
    arrDate.append(date)

    height = new['height'].iloc[0]
    arrHeight.append(height)

    width = new['width'].iloc[0]
    arrWidth.append(width)

    siteName = new['siteName'].iloc[0]
    arrSiteName.append(siteName)

    clientX = new['clientX'].iloc[0]
    arrClientX.append(clientX)

    clientY = new['clientY'].iloc[0]
    arrClientY.append(clientY)

    new2 = (len(new.loc[new['type']=='mouseover']))
    arrMouseover.append(new2)

    new3 = (len(new.loc[new['type'] =='mouseout']))
    arrMouseout.append(new3)

    new4 = (len(new.loc[new['type'] =='click']))
    arrClick.append(new4)

    new5 = (len(new.loc[new['type'] =='blur']))
    arrBlur.append(new5)

    new6 = (len(new.loc[new['type'] =='focus']))
    arrFocus.append(new6)

    new7 = (len(new.loc[new['type'] =='keydown']))
    arrKeydown.append(new7)
    
    counted = new2 + new3 + new4 + new5 + new6 + new7
    count = arrCount.append(counted)

session['count_mouseover'] = arrMouseover
session['count_mouseout'] = arrMouseout
session['count_click'] = arrClick
session['count_blur'] = arrBlur
session['count_focus'] = arrFocus
session['count_keydown'] = arrKeydown
session['count_all'] = arrCount
session['duration'] = arrDate
session['height'] = arrHeight
session['width'] = arrWidth
session['siteName'] = arrSiteName
session['clientX'] = arrClientX
session['clientY'] = arrClientY

###### Iterationen sind einfacher in Typescript. Entschuldigung für den Code, es war leider keine Zeit mehr zum refactoren, aber es funktioniert.

#### 1.3 Alle Sessions die eine Duration über 2000 sec, kürzer als 5 sec und weniger oder nur ein focus Event haben werden aus dem Datensatz gelöscht

In [7]:
session = session.loc[(session['duration'] < 2000) & (session['duration'] > 5) & (session['count_focus'] >= 1)]